# Data set up 

## Load existing dataset

In [1]:
import pandas as pd
df = pd.read_csv('tournesol_scores_above_20_2024-03-15.csv')

## Create dataset

Download videos scores

In [12]:
import io
import zipfile

import requests
import pandas as pd

response = requests.get("https://api.tournesol.app/exports/all")
zip_file = zipfile.ZipFile(io.BytesIO(response.content))
collective_scores = pd.read_csv(zip_file.open("collective_criteria_scores.csv"))

Requests youtube metadata : publication date, title and channel

In [14]:
from dataset import build_dataset
df = build_dataset(collective_scores) #only keeps videos with a tournesol score above 20

Save the dataframe

In [16]:
import datetime
df.to_csv('tournesol_scores_above_20_'+datetime.date.today().isoformat()+'.csv')


# Uniform Recommendation

Test parameters

In [17]:
bundle_size = 4
recent_videos_proportion = 0.75

recent_videos_to_sample = int(bundle_size * recent_videos_proportion)
old_videos_to_sample = int(bundle_size * (1 - recent_videos_proportion))

recent_videos_max_age_in_days = 21

Computes the age of each video

In [18]:
import datetime

from utils import get_age_in_days

today = datetime.datetime.today()
df['age_in_days'] = df.apply(lambda x: get_age_in_days(x, today), axis="columns")

Uniformly sample recent and old videos

In [21]:
recent_videos_sample = df.loc[df['age_in_days']<=21].sample(n=recent_videos_to_sample, replace=False)
old_videos_sample = df.loc[df['age_in_days']<=21].sample(n=old_videos_to_sample, replace=False)

uniform_sample = pd.concat([recent_videos_sample, old_videos_sample])

Download thumbnails in the thumbnails directory

In [22]:
import io

import requests
import ipywidgets as widgets

thumbnail_list = []

for id in uniform_sample['video']:
    thumbnail_url = "https://i.ytimg.com/vi/" + id + "/mqdefault.jpg"
    response = requests.get(thumbnail_url)
    open('thumbnails/'+id+'.jpg', 'wb').write(response.content)

Display images

In [23]:
green_box_layout = widgets.Layout()
green_box_layout.width = '320px'
green_box_layout.height = '180px'
green_box_layout.border = '2px solid green'

hbox_layout = widgets.Layout()
hbox_layout.width = '100%'
hbox_layout.justify_content = 'space-around'

def make_box_for_grid(image_widget, fit):
    """
    Make a VBox to hold caption/image for demonstrating
    option_fit values.
    """
    # Make the caption
    if fit is not None:
        fit_str = "'{}'".format(fit)
    else:
        fit_str = str(fit)
        
    h = widgets.HTML(value='' + str(fit_str) + '')

    # Make the green box with the image widget inside it
    boxb = widgets.Box()
    boxb.layout = green_box_layout
    boxb.children = [image_widget]
    
    # Compose into a vertical box
    vb = widgets.VBox()
    vb.layout.align_items = 'center'
    vb.children = [h, boxb]
    return vb

image_margin = '0 0 0 0'

caption_size = 'h4'

In [24]:
boxes = []
for id in uniform_sample['video']:
    file = open('thumbnails/'+id+'.jpg', 'rb')
    image = widgets.Image(value=file.read())
    image.layout.object_fit = 'contain'
    image.layout.margin = image_margin

    boxes.append(make_box_for_grid(image, uniform_sample.loc[uniform_sample['video']==id, ['title', 'channel']]))

vb = widgets.VBox()
h = widgets.HTML(value='<{size}>Examples of <code>object_fit</code> with large image</{size}>'.format(size=caption_size))
vb.layout.align_items = 'center'
hb = widgets.HBox()
hb.layout = hbox_layout
hb.children = boxes

vb.children = [h, hb]
vb

